# Joe Stanley
### ECE523 - EXAM1

In [1]:
# Import Necessary Libraries
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate as tab
import electricpy as ep
from electricpy.constants import *

![P-1-1.png](P-1-1.png)

![P-1-2.png](P-1-2.png)

### Solution:

***a)***
For loads described only by their positive sequence impedance, it is common to assume the negative sequence impedance as equal (and opposite where applicable in transformer phase shifts) to the positive sequence impedance, and it is also common to assume the zero sequence impedance as three times the positive sequence. That is to say: $$Z_2 \approx Z_1 \qquad Z_0 \approx 3\cdot Z_1$$ For fault analysis, short-circuit MVA or short-circuit current are also valuable pieces of information. Power flow information could be calculated at a steady-state value for unfaulted conditions using the positive sequence impedance to evaluate the power-flow results and treating the "load" as an infinite bus whose voltage magnitude and angle was fixed.

***b)***
Treating the load as constant impedance load would ultimately remove the fixation of voltage magnitude and angle for the "load", however, it would largely leave the impedance relationships unchanged.

***c)***
Treating the load as a constant current load may allow the assumption that the load's impedance will not affect the fault analysis, and that the fault studies may be conducted with impedance matrices that do not include the "load". The "load" current may not be neglected, however. Instead, the load current (being fixed) would be added to the fault current and any other relational power flow currents in much the same way that power-flow currents are treated in textbook-fault studies.

***d)***
To fully account for a fixed power (or constant power) load, the solution method of solution should incorporate sequence component fault analysis prior to a power-flow calculation that includes the fictitious fault bus, accounting for the power consumption for the fault. This will allow for proper accounting of both the fault currents and the constant power currents, which will (inevitably) be affected by the fault's impact on bus voltages.

***e)***
To account for a load that incorporates 40% constant power consumption, 40% constant current consumption, and 20% constant impedance would require decomposing the single load into three discrete "component" loads that represent each type of load characteristic. This would allow the composition of the various methods previously described to effectively represent the system.

![P-2-1.png](P-2-1.png)

![P-2-2.png](P-2-2.png)

In [2]:
# Define Per-Unit Terms
Sbase = 100*M
VbaseT = 230*k
VbaseG = 24*k
ZbaseT = ep.zpu(Sbase,VLL=VbaseT)
ZbaseG = ep.zpu(Sbase,VLL=VbaseG)
IbaseT = ep.ipu(Sbase,VLL=VbaseT)
IbaseG = ep.ipu(Sbase,VLL=VbaseG)

# Define Sequence Impedances
ZS11 = 0.1J
ZS10 = 0.1J
XFM1 = 0.2J
XFM0 = 0.2J
ZS21 = 0.2J
ZS20 = 0.35J
ZS31 = 0.2J
ZS30 = 0.7J
ZL11 = 0.01+0.15J
ZL10 = 0.06+0.5J
ZL21 = 0.05+0.5J
ZL20 = 0.20+2.0J
ZL31 = 0.02+0.2J
ZL30 = 0.07+0.7J
ZL41 = 0.02+0.2J
ZL40 = 0.07+0.7J    

# Define Function to Provide Impedance Matrices
def zmatrix(ZL1=True,ZL2=True,ZL3=True,ZL4=True,S2=True,S3=True,verbose=False):
    # Define Pos/Neg Seq Y-Bus Matrix
    y12 = np.array([
        [1/(ZS11+XFM1)+int(ZL1)/ZL11+int(ZL2)/ZL21, -int(ZL1)/ZL11, -int(ZL2)/ZL21],
        [-int(ZL1)/ZL11, int(ZL1)/ZL11+1/(int(ZL3)*ZL31+int(ZL4)*ZL41)+int(S2)/ZS21, -1/(int(ZL3)*ZL31+int(ZL4)*ZL41)],
        [-int(ZL2)/ZL21, -1/(int(ZL3)*ZL31+int(ZL4)*ZL41), int(ZL2)/ZL21+1/(int(ZL3)*ZL31+int(ZL4)*ZL41)+int(S3)/ZS31],
    ])
    # Define Zero Seq Y-Bus Matrix
    y0 = np.array([
        [1/(ZS10+XFM0)+int(ZL1)/ZL10+int(ZL2)/ZL20, -int(ZL1)/ZL10, -int(ZL2)/ZL20],
        [-int(ZL1)/ZL10, int(ZL1)/ZL10+1/(int(ZL3)*ZL30+int(ZL4)*ZL40)+int(S2)/ZS20, -1/(int(ZL3)*ZL30+int(ZL4)*ZL40)],
        [-int(ZL2)/ZL20, -1/(int(ZL3)*ZL30+int(ZL4)*ZL40), int(ZL2)/ZL20+1/(int(ZL3)*ZL30+int(ZL4)*ZL40)+int(S3)/ZS30],
    ])
    # Calculate Z-Bus Matrices
    zbus1 = np.linalg.inv(y12)
    zbus2 = np.linalg.inv(y12)
    zbus0 = np.linalg.inv(y0)
    # Print if Necessary
    if verbose:
        print("\nPositive Sequence Z-Bus:\n",tab(np.asarray(np.around(zbus1,3),dtype=str),tablefmt="fancy_grid"),sep='')
        print("\nNegative Sequence Z-Bus:\n",tab(np.asarray(np.around(zbus2,3),dtype=str),tablefmt="fancy_grid"),sep='')
        print("\nZero Sequence Z-Bus:\n",tab(np.asarray(np.around(zbus0,3),dtype=str),tablefmt="fancy_grid"),sep='')
    # Return Results
    return(zbus1,zbus2,zbus0)

# Demonstrate Validity
print("Z-Bus, All On-Line")
x = zmatrix(verbose=True)

Z-Bus, All On-Line

Positive Sequence Z-Bus:
╒═════════════════╤═════════════════╤═════════════════╕
│ (0.003+0.129j)  │ (-0+0.069j)     │ (-0.002+0.045j) │
├─────────────────┼─────────────────┼─────────────────┤
│ (-0+0.069j)     │ (0.002+0.111j)  │ (-0.002+0.044j) │
├─────────────────┼─────────────────┼─────────────────┤
│ (-0.002+0.045j) │ (-0.002+0.044j) │ (0.003+0.126j)  │
╘═════════════════╧═════════════════╧═════════════════╛

Negative Sequence Z-Bus:
╒═════════════════╤═════════════════╤═════════════════╕
│ (0.003+0.129j)  │ (-0+0.069j)     │ (-0.002+0.045j) │
├─────────────────┼─────────────────┼─────────────────┤
│ (-0+0.069j)     │ (0.002+0.111j)  │ (-0.002+0.044j) │
├─────────────────┼─────────────────┼─────────────────┤
│ (-0.002+0.045j) │ (-0.002+0.044j) │ (0.003+0.126j)  │
╘═════════════════╧═════════════════╧═════════════════╛

Zero Sequence Z-Bus:
╒═════════════════╤═════════════════╤═════════════════╕
│ (0.005+0.205j)  │ (-0.004+0.081j) │ (-0.003+0.06j)  │
├──────────

In [4]:
help(ep.fault.phs3mvasc)

Help on function phs3mvasc in module electricpy.fault:

phs3mvasc(Vth, Zseq, Rf=0, Sbase=1)
    Three-Phase MVA Short-Circuit Calculator
    
    Calculator to evaluate the Short-Circuit MVA of a three-phase fault given the system
    parameters of Vth, Zseq, and an optional Rf. Uses the formula as follows:
    
    .. math:: MVA_{sc} = \frac{\left|V_{th}**2\right|}{|Z_1|} * Sbase
    
    Parameters
    ----------
    Vth:        complex
                The Thevenin-Equivalent-Voltage
    Zseq:       list of complex
                Tupple of sequence reactances as (Z0, Z1, Z2)
    Rf:         complex, optional
                The fault resistance, default=0
    Sbase:      real, optional
                The per-unit base for power. default=1
    
    Returns
    -------
    MVA:        real
                Three-Phase Short-Circuit MVA.

